In [1]:
import numpy as np
import itertools
import math
from tqdm.notebook import tqdm
from copy import copy
from collections import deque

In [2]:
class Point():
    def __init__(self, x):
        self.cord = np.array(x)
    def dist(self, other):
        return np.sqrt(np.sum(np.square(self.cord - other.cord)))
    def get_dimension(self):
        return self.cord.shape[0]

In [3]:
def SimpleSearch(vertexes, target_vertex, num):
    dist_mas = np.zeros(vertexes.shape[0])
    for i in range(vertexes.shape[0]):
        dist_mas[i] = target_vertex.dist(vertexes[i])
    return dist_mas.argpartition(num)[:num]

In [4]:
def CalcSumDist(vertexes, target_vertex):
    dist = 0.0
    for v in vertexes:
        dist += v.dist(target_vertex)
    return dist

In [5]:
class ListMin():
    
    def __init__(self):
            self.size = 0
            self.min = None
            self.deq = deque()
            
    def add_first(self, e):
        self.size = 1
        self.deq.appendleft(e[1])
        self.min = e[0]
        return
    
    def add(self, e):
        self.size += 1
        if self.min > e[0]:
            self.min = e[0]
            self.deq.appendleft(e[1])
        else:
            self.deq.append(e[1])
        return
    
    def add_no_min_no_size(self, e):
        self.deq.append(e)
        
    def get_min(self):
        return self.min
    
    def size(self):
        return self.size
    
    def change_min(self, e):
        self.size += 1
        self.min = e[0]
        self.deq.appendleft(e[1])
        return
    
    def get_elements(self):
        return [i for i in self.deq]

In [6]:
class HNSW():
    
    def __init__(self, vertexes, number_of_candidates = 20, maximum_number_neighbors = None, maximum_number_neighbors0 = None, control_layers = None, type_of_search = None, extension_in_search = True, keep_pruned = False):
        if control_layers is None:
            self.mL = 1 / np.log(number_of_candidates)
        else:
            self.mL = control_layers
        if maximum_number_neighbors is None:
            self.maximum_number_neighbors = 2 * number_of_candidates
        else:
            self.maximum_number_neighbors = maximum_number_neighbors
        if maximum_number_neighbors is None:
            self.maximum_number_neighbors0 = 2 * number_of_candidates
        else:
            self.maximum_number_neighbors0 = maximum_number_neighbors0
        if type(vertexes) == type(list()):
            self.vertexes = vertexes
        else:
            self.vertexes = vertexes.tolist()
        if type_of_search is None:
            if len(vertexes):
                self.find_closest = self.__simple_select__ if vertexes[0].get_dimension() < 10 else self.__heuristic_select__
            else:
                self.find_closest = self.__simple_select__
        else:
            self.find_closest = self.__simple_select__ if type_of_search == 'simple' else self.__heuristic_select__
        self.extension_in_search = extension_in_search
        self.keep_pruned = keep_pruned
        self.index_per_layer = list()
        self.matrix = list()
        self.layers = list()
        self.enter_point = None
        self.toplayer = 0
        self.number_of_candidates = number_of_candidates
        if len(self.vertexes) > 0:
            self.__insert_first__(0)
        for i in tqdm(range(1, len(self.vertexes))):
            self.__insert_private__(i, self.number_of_candidates, self.maximum_number_neighbors, self.maximum_number_neighbors0)
        
    def __insert_first__(self, q):
        self.layers.append([q])
        self.enter_point = q
        self.matrix.append([[]])
        self.index_per_layer.append(dict())
        self.index_per_layer[0][q] = 0
    
    def __get_new_level__(self):
        return -np.log(np.random.uniform()) * self.mL
    
    def __simple_select__(self, vertexes, target, n, level):
        s = set()
        for i in vertexes:
            s.add((self.vertexes[target].dist(self.vertexes[i]), i))
        return [i[1] for i in itertools.islice(s, n)]
    
    def __heuristic_select__(self, vertexes, target, n, level):
        res = ListMin()
        w = set([(self.vertexes[i].dist(self.vertexes[target]), i) for i in vertexes])
        w_ind = set([i for i in vertexes]) if self.extension_in_search else None
        if self.extension_in_search:
            for i in vertexes:
                for j in self.matrix[level][self.index_per_layer[level][i]]:
                    if j not in w_ind:
                        w.add((self.vertexes[target].dist(self.vertexes[j]), j))
                        w_ind.add(j)
        discarded = set()
        del w_ind
        while w:
            el = w.pop()
            dist_e = el[0]
            e = el[1]
            if res.size == 0:
                res.add_first(el)
            else:
                minimum = res.get_min()
                if minimum > dist_e:
                    res.change_min(el)
                    if res.size >= n:
                        break
                else:
                    discarded.add(el)
        if self.keep_pruned:
            while res.size < n and discarded:
                res.add_no_min_no_size(discarded.pop()[1])
        return res.get_elements()
        
    def __add_conections__(self, vertexes, q, M_max, l_now):
        for i in vertexes:
            ind = self.index_per_layer[l_now][i]
            if len(self.matrix[l_now][ind]) < M_max:
                self.matrix[l_now][ind].append(q)
            else:
                self.matrix[l_now][ind] = self.find_closest(self.matrix[l_now][ind] + [q], i, M_max, l_now)
        ind = self.index_per_layer[l_now][q]
        self.matrix[l_now][ind] = vertexes
        return
            
        
    def __insert_private__(self, q, M, M_max, M_max0):
        l = self.toplayer
        ep = [self.enter_point]
        new_l = math.floor(self.__get_new_level__())
        for l_now in range(l, new_l, -1):
            ep = self.__search_layer__(q, ep, 1, l_now)

        for l_now in range(min(l, new_l), -1, -1):
            w = self.find_closest(self.__search_layer__(q, ep, M, l_now), q, M, l_now)
            self.index_per_layer[l_now][q] = len(self.layers[l_now])
            self.layers[l_now].append(q)
            self.index_per_layer[l_now][q] = len(self.matrix[l_now])
            self.matrix[l_now].append([])
            self.__add_conections__(w, q, M_max if l_now != 0 else M_max0, l_now)
            ep = w
        if new_l > self.toplayer:
            for i in range(new_l - self.toplayer):
                self.layers.append([q])
                self.matrix.append([[]])
                self.index_per_layer.append(dict())
                self.index_per_layer[-1][q] = 0
            self.toplayer = new_l
            self.enter_point = q
        return
    
    def __search_layer__(self, q, ep, ef, l):
        candidates = set([(self.vertexes[i].dist(self.vertexes[q]), i) for i in ep])
        visited = set(ep)
        w = set([(-self.vertexes[i].dist(self.vertexes[q]), i) for i in ep])
        while bool(candidates):
            c = candidates.pop()
            dist_c = c[0]
            c = c[1]
            w_top = w.pop()
            f = w_top[1]
            dist_f = -w_top[0]
            w.add(w_top)
            if dist_c > dist_f:
                break
            for e in self.matrix[l][self.index_per_layer[l][c]]:
                if e not in visited:
                    visited.add(e)
                    w_top = w.pop()
                    f = w_top[1]
                    dist_f = -w_top[0]
                    w.add(w_top)
                    dist_e = self.vertexes[e].dist(self.vertexes[q])
                    if len(w) < ef:
                        candidates.add((dist_e, e))
                        w.add((dist_e, e))
                    else:
                        if dist_e < dist_f:
                            candidates.add((dist_e, e))
                            w.add((dist_e, e))
                            w.remove(w_top)
        return [i[1] for i in w]
    
    def get_info(self):
        print('total levels', self.toplayer)
        for i in range(self.toplayer + 1):
            print('layer', i, 'elements', len(self.layers[i]))
        return self.index_per_layer, self.layers, self.matrix 
        
    
    def add(self, vertex):
        q = len(self.vertexes)
        self.vertexes.append(vertex)
        self.__insert_private__(self, q, self.number_of_candidates, self.maximum_number_neighbors)
        return
    
    def SearchNeighbors(self, vertex, ef, return_index = True, type_of_search = None, change_number_of_enter = lambda s: s):
        q = len(self.vertexes)
        self.vertexes.append(vertex)
        backup_extension_in_search = self.extension_in_search
        backup_keep_pruned = self.keep_pruned
        self.extension_in_search = True
        self.keep_pruned = True
        ep = [self.enter_point]
        cnt = 1
        for l in range(self.toplayer, -1, -1):
            ep = self.__search_layer__(q, ep, cnt, l)
            cnt += change_number_of_enter(cnt)
            
        w = self.__search_layer__(q, ep, ef, 0)
        res = self.__simple_select__(w, q, ef, 0)
        del self.vertexes[-1]
        self.extension_in_search = backup_extension_in_search
        self.keep_pruned = backup_keep_pruned
        return res if return_index else self.vertexes[res]
            
        

In [11]:
n = 10000
dim = 1000
test_mas = np.array([Point(np.random.uniform(-10000, 10000, dim)) for i in range(n)])

In [12]:
%%time
hnsw = HNSW(test_mas, 20, type_of_search = 'simple')


CPU times: user 1min 20s, sys: 485 ms, total: 1min 20s
Wall time: 1min 20s


In [26]:
dist_1 = list()
dist_2 = list()
for i in tqdm(range(50)):
    test_vertex = Point(np.random.uniform(-10000, 10000, dim))
    index1 = hnsw.SearchNeighbors(test_vertex, 10, type_of_search = 'simple', change_number_of_enter = lambda s: s + 10)
    dist_1.append(CalcSumDist(test_mas[index1], test_vertex))

    index2 = SimpleSearch(test_mas, test_vertex, 10)
    dist_2.append(CalcSumDist(test_mas[index2], test_vertex))
    if len(index1) < len(index2):
        print('can\'t find 10 neigbours')

In [27]:
dist_1 = np.array(dist_1)
dist_2 = np.array(dist_2)
print('sum')
print('HNSW', np.sum(dist_1), 'LinearSearch', np.sum(dist_2))
print('average difference in % between HNSW distance and LinearSearch distance')
print(np.sum(dist_1) /  np.sum(dist_2) - 1)

sum
HNSW 128744480.77222222 LinearSearch 121473075.5699372
average difference in % between HNSW distance and LinearSearch distance
0.05986022143728942


In [28]:
%%time
for i in tqdm(range(100)):
    test_vertex = Point(np.random.uniform(-10000, 10000, dim))
    index = hnsw.SearchNeighbors(test_vertex, 10, type_of_search = 'simple', change_number_of_enter = lambda s: s + 10)


CPU times: user 282 ms, sys: 5.22 ms, total: 287 ms
Wall time: 285 ms


In [29]:
%%time
for i in tqdm(range(100)):
    test_vertex = Point(np.random.uniform(-10000, 10000, dim))
    index = SimpleSearch(test_mas, test_vertex, 10)


CPU times: user 9.37 s, sys: 44.6 ms, total: 9.42 s
Wall time: 9.42 s


More than 10 times faster than simple linear search